### Baseline data for COSVF in CALVIN

This script prepares files necessary to instatiate a Carryover Storage Value Function (COSVF) annual limited foresight model of CALVIN. The links file is the export for the first water year (1922) with debug links added from Calvin Network tools.

1. ``links.csv``: 


    Network for the first water year in the period of analysis.
  
    CSV file with column headers: ``i,j,k,cost,amplitude,lower_bound,upper_bound``

2. ``cosvf-params.csv`` 
    
    A table of minimum and maximum marginal cost penalties for quadratic carryover penalty curves on surface water reservoirs and linear penalties on groundwater reservoirs. **The values are not optimized. They are just stand-in values to be replaced by the evolutionary optimization run.**

    CSV file with column headers: ``r,param,value``

3.  ``r-dict.json``: 
    
    A dictionary of reservoirs in the network with penalty properties. 

    Type 2 (linear penalty) reservoirs must be ordered prior to the Type 1 (quadratic) reservoirs. This is a limitation of imposed by the evolutionary algorithm search of the parameters. For each reservoir with an EOP penalty, an index attribute ``cosvf_param_index`` points to the row-index (pythonic zero-indexed) of the list of reservoirs in ``cosvf-params.csv``.

    Dictionary structure:

        {"<reservoir id (e.g. SR_DNP)>":
        
            {
              "eop_init": "(float) initial (October 1) storage level",
              
              "lb": "(float) minimum (end-of-September) storage level",
              
              "ub": "(float) maximum (end-of-September) carryover capactiy",
              
              "type": "(int) 0:none; 1:quadratic; or 2:linear",
              
              "cosvf_param_index": "(list) index to cosvf_params.csv row (pythonic zero-indexed)",
              
              "k_count": "(int) number of piecewise links to fit for quadratic COSVF",
            }

        }

4.  ``inflows.csv``: 
    
    External inflows for every monthly time step over the period of analysis to run. 

    CSV file with column headers ``date, j, flow_taf``

5.  ``variable-constraints.csv``: 
    
    Links that have variable year-to-year upper and/or lower bounds.

    CSV file with column headers ``date,i,j,k,lower_bound,upper_bound``




In [ ]:
import os,sys
import json
import numpy as np
import pandas as pd

In [ ]:
# Load pyVIN
calvin_dir = os.path.abspath('../../calvin')
if str(calvin_dir)!=sys.path:
    sys.path.append(calvin_dir)
from calvin import CALVIN, cosvfea
from calvin import *

In [ ]:
# perfect forsight model directory
pf_dir = '../../calvin/my-models/calvin-pf'

#limited forsight model directory
lf_dir = '../../calvin/my-models/calvin-lf'

In [ ]:
#  %% load perfect foresight links
pf_links = pd.read_csv(os.path.join(pf_dir,'links82yr.csv'))

In [ ]:
# remove debug nodes
pf_links = pf_links.loc[
    (~pf_links.i.str.contains('DBUG')) & (~pf_links.j.str.contains('DBUG'))]

In [ ]:
# create a column for edges (w/o dates)
pf_links.insert(0, 'edge', 
    value=pf_links.i.str.split('.').str[0]+'_'+pf_links.j.str.split('.').str[0])
pf_links.insert(1,'i_node',
    value=pf_links.i.map(lambda x: x.split('.')[0]))
pf_links.insert(2,'j_node',
    value=pf_links.j.map(lambda x: x.split('.')[0]))
pf_links.insert(0, 'year',
    value=pd.DatetimeIndex(pf_links.i.str.split('.').str[1]).year)
pf_links.insert(1, 'month',
    value=pd.DatetimeIndex(pf_links.i.str.split('.').str[1]).month)
pf_links.insert(0, 'date',
    value=pd.DatetimeIndex(pf_links.i.str.split('.').str[1]))

In [ ]:
# %% Inflows
# Extract monthly inflows to csv
# load inflows 
inflow_qwry = pf_links.loc[(pf_links.i.str.contains('INFLOW'))]
# split j to node and date
inflows = inflow_qwry['j'].str.split('.',expand=True)
inflows.columns = ['j','date']
inflows['date'] = pd.DatetimeIndex(inflows['date'])
inflows.set_index('date',inplace=True)
# get inflow values
inflows.insert(1,'flow_taf', value = inflow_qwry['lower_bound'].values)

In [ ]:
# %%  save out inflows output
inflows.to_csv(os.path.join(lf_dir,'inflows.csv'))

In [ ]:
# %% Variable Constraints
# Query out upper and lower bounds that change from year to year and export to csv
def get_variable_range(links,column):
    variable_links = links.groupby(
        ['edge','month','k'])[column].max().subtract(
        links.groupby(['edge','month','k'])[column].min())
    return(pd.DataFrame(variable_links.iloc[np.where(variable_links>0)]))

In [ ]:
variable_links = pf_links.loc[
    (~pf_links.i.str.contains('INFLOW')) &
    (~pf_links.i.str.contains('INITIAL')) ]
variable_lb = get_variable_range(variable_links,'lower_bound')
variable_ub = get_variable_range(variable_links,'upper_bound')
variable_min_max = variable_ub.join(variable_lb,how='outer')
variable_min_max.head()
# query the constraints that were found to vary
variable_constraints = pf_links.loc[pf_links['edge'].isin(
    variable_min_max.index.get_level_values(0).unique())]
# subset storage variable constraints for September only
variable_constraints_storages = variable_constraints.loc[
    (variable_constraints.i_node==variable_constraints.j_node) &
    (variable_constraints.month!=9)]
# subset down to the k=0 link
variable_constraints_storages_k0 = variable_constraints_storages.loc[variable_constraints_storages.k==0]
# get the upper bound on storage variable links
variable_constraints_storages_k0ub = variable_constraints_storages.groupby(['i','j'],as_index=False)['upper_bound'].sum()
# join to k0 link set and set the actual upper bound 
variable_constraints_storages_k0 = variable_constraints_storages_k0.merge(variable_constraints_storages_k0ub, on=['i','j'],suffixes=('','_join'))
variable_constraints_storages_k0['upper_bound'] = variable_constraints_storages_k0['upper_bound_join'] 
# set the lower bound on Shasta and Clair Engle to the default constant values
variable_constraints_storages_k0.loc[variable_constraints_storages_k0.edge=='SR_SHA_SR_SHA','lower_bound'] = 650
variable_constraints_storages_k0.loc[variable_constraints_storages_k0.edge=='SR_CLE_SR_CLE','lower_bound'] = 500
# remove storage variable constraints from main variable constraints
variable_constraints = variable_constraints.loc[
    variable_constraints.i_node!=variable_constraints.j_node]
# add back in the variable storage constraints for all other months than September
variable_constraints = variable_constraints.append(variable_constraints_storages_k0)   

In [ ]:
# %% save out variable constraints output
variable_constraints = variable_constraints[['date','i','j','k','lower_bound','upper_bound']]
variable_constraints.to_csv('variable-constraints.csv',index=False)

In [ ]:
# %% Reservoirs
# List of reservoirs
r_list = pf_links.loc[(pf_links.i_node.str.startswith('INITIAL'))].j_node.unique()

In [ ]:
# Reservoirs identified as COSVF canditates (Type 1)
r_type1 = ['SR_BER','SR_BUC','SR_BUL','SR_CLE','SR_CLK_INV','SR_CMN','SR_DNP',
            'SR_EBMUD','SR_FOL','SR_HTH','SR_ISB','SR_LL_ENR','SR_LVQ','SR_MCR',
            'SR_MIL','SR_NHG','SR_NML','SR_ORO','SR_PAR','SR_PNF','SR_RLL_CMB',
            'SR_SHA','SR_SNL','SR_SFAGG','SR_GNT','SR_WHI']

r_type2 = ['GW_01', 'GW_02', 'GW_03','GW_04', 'GW_05', 'GW_06','GW_07',
           'GW_08', 'GW_09', 'GW_10', 'GW_11', 'GW_12', 'GW_13', 'GW_14', 'GW_15',
           'GW_16', 'GW_17', 'GW_18', 'GW_19', 'GW_20', 'GW_21',
           'GW_AV', 'GW_CH', 'GW_EW', 'GW_IM', 'GW_MJ', 'GW_MWD',
           'GW_OW', 'GW_SBV', 'GW_SC', 'GW_SD', 'GW_VC']

In [ ]:
# reservoir dictionary for calvin limited foresight run
r_dict = dict()
i = 0
for r in r_list:
    # initial storage value
    initial_storage = pf_links.loc[
        (pf_links.i_node=='INITIAL') & (pf_links.j_node==r)].lower_bound
    # lower bound on carryover
    lb_9 = pf_links.loc[
        (pf_links.i_node==r) & (pf_links.j_node==r) & 
        (pf_links.k==0) & (pf_links.month==9)].lower_bound.min()
    # upper bound on carryover from first year
    ub_9 = pf_links.loc[
        (pf_links.i_node==r) & (pf_links.j_node==r) & 
        (pf_links.month==9) & (pf_links.year==1922)].upper_bound.sum()
    # check COSVF Type 1 to index COSVF param
    if r in r_type1:
        r_type, cosvf_param_index, k_count, i = 1, [i,i+1], 15, i+2
    elif r in r_type2:
        r_type, cosvf_param_index, k_count, i = 2, i, 2, i+1
    else:
        r_type, cosvf_param_index, k_count = 0, None, 1
    # add to reservoir dictionary
    r_dict[r] = dict([
        ('eop_init',initial_storage.values[0]),
        ('lb',lb_9),
        ('ub',ub_9),
        ('type',r_type), 
        ('cosvf_param_index',cosvf_param_index),
        ('k_count',k_count)])

In [ ]:
# %% save out the reservoir dictionary to json file
with open(os.path.join(lf_dir,'r-dict.json'), 'w') as json_file:
    json.dump(r_dict, json_file, 
        sort_keys=False, indent=4, separators=(',', ': '))

In [ ]:
# %% Create default COSVF params
param=['pmin','pmax']
rtype1_list = list({key: value for key, value in r_dict.items() if value['type'] == 1}.keys())
rtype2_list = list({key: value for key, value in r_dict.items() if value['type'] == 2}.keys())
pos_r_list = rtype2_list + list(np.repeat(rtype1_list, len(param)))
cosvf_pminmax = pd.DataFrame({'value':
    list(np.repeat([-1e2], len(rtype2_list))) + list(np.tile([-1e2, -7e2], len(rtype1_list)))})
cosvf_pminmax.insert(0,'r',value=pos_r_list)
cosvf_pminmax.insert(1,'param',value=list(['p'] * len(rtype2_list) + param * len(rtype1_list)))

In [ ]:
# %% save out default COSVF params
cosvf_pminmax.to_csv(os.path.join(lf_dir,'cosvf-params.csv'),index=False)

In [ ]:
links = pd.read_csv('links_default.csv')
links.insert(0, 'edge', 
    value=links.i.str.split('.').str[0]+'_'+links.j.str.split('.').str[0])
links.insert(1,'i_node',
    value=links.i.map(lambda x: x.split('.')[0]))
links.insert(2,'j_node',
    value=links.j.map(lambda x: x.split('.')[0]))

In [ ]:
links_storages = links.loc[(links.i_node==links.j_node) & (links.i_node.isin(r_type1))]
# subset down to the k=0 link
links_storages_k0 = links_storages.loc[links_storages.k==0]
# default all persuasions to -0.02 $/af
links_storages_k0['cost'] = -0.02
# get the upper bound on storage variable links
links_storages_k0ub = links_storages.groupby(['i','j'],as_index=False)['upper_bound'].sum()
# join to k0 link set and set the actual upper bound 
links_storages_k0 = links_storages_k0.merge(links_storages_k0ub, on=['i','j'],suffixes=('','_join'))
links_storages_k0['upper_bound'] = links_storages_k0['upper_bound_join'] 
# set the lower bound on Shasta and Clair Engle to the default constant values
links_storages_k0.loc[links_storages_k0.edge=='SR_SHA_SR_SHA','lower_bound'] = 650
links_storages_k0.loc[links_storages_k0.edge=='SR_CLE_SR_CLE','lower_bound'] = 500
# remove storage variables from main links 
r_type1_concat = [x+'_'+x for x in r_type1]
links = links.loc[~(links.edge.isin(r_type1_concat))]
# add back in modified storage links to the main links
links = links.append(links_storages_k0)   

In [ ]:
# %%
links[['i','j','k','cost','amplitude','lower_bound','upper_bound']].to_csv('links.csv',index=False)